## Chairman's Census

- **FFXIV**: The Chairman, Excalibur
- **Mastodon/Fediverse**: @jimpjorps@knzk.me
- **Twitter**: @jimpjorps
- **Discord**: jimpjorps#0001 
 

In [ ]:
# Necessary packages

%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

import matplotlib.pyplot as plt
import numpy as npt
import requests
from collections import Counter
import pandas as pd
from tqdm import tqdm

In [ ]:
# Put your XIVAPI key here

xivapi_key = "api_key_goes_here"

In [ ]:
# Look up the FC's ID number
# If this step doesn't work because you have a common or hard-to-search-for FC name,
# set fc_id directly

url_base = "https://xivapi.com"
fc_name = "Please Calm Down"
server_name = "Omega"
payload = {
    "key": xivapi_key,
    "name": fc_name,
    "server": server_name
}
r = requests.post(url_base + "/freecompany/search", json=payload)
fc_id = r.json()['Results'][0]['ID']
fc_id

In [ ]:
# Retrieve the character IDs of everyone in this FC
# Show the first 10 IDs to check

payload = {
    "key": xivapi_key,
    "data": "FCM",
    "columns": "FreeCompanyMembers.data.*.ID"
}
r = requests.post(url_base + "/freecompany/" + fc_id, json=payload)
members_data = r.json()['FreeCompanyMembers']['data']
members_id = [i['ID'] for i in members_data]

members_id[0:9]

In [ ]:
# Define the lists of races, tribes and genders to convert the provided 
# indices into readable text, as well as to make using pandas easier

census_list = []
races = [
    "Unknown",
    "Hyur",
    "Elezen",
    "Lalafell",
    "Miqote",
    "Roegadyn",
    "Au Ra"
]
tribes = [
    "Unknown",
    "Midlander",
    "Highlander",
    "Wildwood",
    "Duskwight",
    "Plainsfolk",
    "Dunesfolk",
    "Seeker",
    "Keeper",
    "Sea Wolf",
    "Hellsguard",
    "Raen",
    "Xaela"
]
genders = [
    "Unknown",
    "M",
    "F"
]

In [ ]:
# Request the race/tribe/gender indices for each character
# Make a list of tuplets

census_list = []

payload = {
    "key": xivapi_key,
    "columns": "Character.Race,Character.Tribe,Character.Gender"
}

for id in tqdm(members_id):
    r = requests.post(url_base + "/character/" + str(id), json=payload)
    datum = [
        races[int(r.json()['Character']['Race'] or 0)],
        tribes[int(r.json()['Character']['Tribe'] or 0)],
        genders[int(r.json()['Character']['Gender'] or 0)]
    ]
    census_list.append(datum)
    
census_list

In [ ]:
# Turn the list of tuples into a pandas DataFrame
# Make a pivot table that summarizes by count

df = pd.DataFrame(
    census_list,
    columns=["Race", "Tribe", "Gender"]
)
pt = pd.pivot_table(
    df, 
    index=['Race','Tribe'], 
    columns=['Gender'],
    aggfunc=len,
    fill_value=0,
)
ax = pt.plot.barh(
    figsize=[12, 6],
    stacked=True,
    table=False,
    width=0.75,
    title=fc_name
)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
for p in ax.patches:
    ax.annotate(str(p.get_width()), (p.get_x() + p.get_width(), p.get_y()), xytext=(4, 6), textcoords='offset points')
ax.set_facecolor('white')
ax.patch.set_alpha(1)

In [ ]:
# Generate a pivot table showing the demographic breakdown with subtotals

pt = pd.pivot_table(df, 
                    index=['Race','Tribe'], 
                    columns=['Gender'],
                    aggfunc=len,
                    fill_value=0,
                    margins=True
                   )
pt